In [ ]:
%load_ext autoreload
%autoreload 2

# Open3D PreProcessing

In [ ]:
from lib.data.metainfo import MetaInfo
from lib.visualize.open3d import (
    visualize_object,
    visualize_pointcloud,
    visualize_sdf_slice,
)
from lib.visualize.image import image_grid
from lib.data.preprocess import PreprocessSDF, PreprocessMesh, PreprocessSiamese


data_dir = "/home/borth/sketch2shape/data/shapenet_chair_16"
metainfo = MetaInfo(data_dir=data_dir)
obj_id = metainfo.obj_ids[0]

## Load And Normalize

In [ ]:
# watertight + laplacian smoothing
mesh = PreprocessMesh(
    data_dir=data_dir,
    laplacian_num_iters=1,
    resolution=30000,
    smoothing=True,
)
normalized_mesh = mesh.preprocess(obj_id)

# raw mesh
# mesh = PreprocessMesh(data_dir=data_dir, laplacian_num_iters=2, resolution=20000)
# normalized_mesh = mesh.normalize_mesh(metainfo.load_mesh(obj_id))

metainfo.save_normalized_mesh(obj_id=obj_id, mesh=normalized_mesh)
visualize_object(normalized_mesh)

## Sphere Tracing and Normals Map

In [ ]:
azims = [0, 20, 40]
elevs = [-50, -30, -10, 10, 30]
print(len(azims) * len(elevs))
siamese = PreprocessSiamese(
    data_dir=data_dir,
    azims=azims,
    elevs=elevs,
    width=256,
    height=256,
    focal=512,
    t_lower=100,
    t_upper=150,
    aperture_size=3,
    l2_gradient=True,
    # width=64,
    # 0height=64,
    # focal=128,
)
normals, sketches = siamese.preprocess(obj_id=obj_id)

In [ ]:
_ = image_grid(normals, rows=len(azims), cols=len(elevs))

In [ ]:
_ = image_grid(sketches, rows=len(azims), cols=len(elevs))

# Sample SDF 

In [ ]:
preprocess_sdf = PreprocessSDF(data_dir=data_dir)
preprocess_sdf.load(obj_id=obj_id)
points, sdfs = preprocess_sdf.sample_near_surface(100000, scale=5e-02)
visualize_pointcloud(points, sdfs)

In [ ]:
points, sdfs = preprocess_sdf.sample_near_surface(100000, scale=5e-03)
visualize_pointcloud(points, sdfs)

In [ ]:
points, sdfs = preprocess_sdf.sample_unit_ball(100000)
mask = sdfs < 0
visualize_pointcloud(points[mask], sdfs[mask])

In [ ]:
preprocess_sdf = PreprocessSDF(data_dir=data_dir, unit_samples=50000)
sdfs, surface = preprocess_sdf.preprocess(obj_id=obj_id)
visualize_pointcloud(sdfs[:, :3], sdfs[:, 3])

In [ ]:
_ = visualize_sdf_slice(normalized_mesh, dim="x", mask=True)

In [ ]:
_ = visualize_sdf_slice(normalized_mesh, dim="y", mask=True)

In [ ]:
_ = visualize_sdf_slice(normalized_mesh, dim="y", mask=True)